Parsing Scraped Data File into Seperate Eneities

In [12]:
CREATE TABLE [airbnb].[dbo].[hosts]
    (host_id INT PRIMARY KEY
    , host_name NVARCHAR(40)
    , host_since DATETIME2
    , host_location NVARCHAR(100)
    , response_time NVARCHAR(20)
    , response_rate FLOAT
    , acceptance_rate FLOAT
    , is_superhost BIT
    , neighborhood NVARCHAR(30)
    , listings_count INT
    , total_listings_count INT
    , verifications NVARCHAR(150)
    , has_profile_pic BIT
    , identity_verified BIT);
    

INSERT INTO [airbnb].[dbo].[hosts]
    SELECT DISTINCT
    host_id
    , CONVERT(NVARCHAR(40),host_name)
    , host_since
    , CONVERT(NVARCHAR(100),host_location)
    , CONVERT(NVARCHAR(20),host_response_time)
    , CONVERT(FLOAT,CAST(host_response_rate AS VARCHAR(MAX)))
    , CONVERT(FLOAT,CAST(host_acceptance_rate AS VARCHAR(MAX)))
    , CASE
        WHEN CAST(host_is_superhost AS NVARCHAR(1)) = 't'
            THEN 'TRUE'
        ELSE 'FALSE'
      END
    , CONVERT(NVARCHAR(30),host_neighbourhood)
    , CONVERT(INT,CAST(host_listings_count AS VARCHAR(MAX)))
    , CONVERT(INT,CAST(host_total_listings_count AS VARCHAR(MAX)))
    , CONVERT(NVARCHAR(30),host_verifications)
    , CASE
        WHEN CAST(host_has_profile_pic AS NVARCHAR(1)) = 't'
            THEN 'TRUE'
        ELSE 'FALSE'
      END
    , CASE
        WHEN CAST(host_identity_verified AS NVARCHAR(1)) = 't'
            THEN 'TRUE'
        ELSE 'FALSE'
      END
    FROM [airbnb].[dbo].[scraped_listings]

(25690 rows affected)

Total execution time: 00:00:00.818

Similarly, we will need to parse the Listings into its own Entity

In [13]:
CREATE TABLE [airbnb].[dbo].[listings]
    (listing_id INT PRIMARY KEY
    , host_id INT
    , neighbourhood NVARCHAR(100)
    , neighbourhood_group NVARCHAR(100)
    , latitude FLOAT
    , longitude FLOAT
    , property_type NVARCHAR(50)
    , room_type NVARCHAR(50)
    , accommodates INT
    , bathrooms INT
    , bedrooms INT
    , beds INT
    , amenities_concat VARCHAR(MAX)
    , price INT
    , availability_365 INT
    , number_of_reviews INT
    , review_scores_rating FLOAT);
    

INSERT INTO [airbnb].[dbo].[listings]
    SELECT
    Id
    , host_id
    , CAST(neighbourhood_cleansed AS NVARCHAR(100))
    , CAST(neighbourhood_group_cleansed AS NVARCHAR(100))
    , CAST(CAST(latitude AS VARCHAR(MAX)) AS FLOAT)
    , CAST(CAST(longitude AS VARCHAR(MAX)) AS FLOAT)
    , CAST(property_type AS NVARCHAR(50))
    , CAST(room_type AS NVARCHAR(50))
    , CONVERT(INT,CAST(accommodates AS VARCHAR(MAX)))
    , CONVERT(INT,CAST(bathrooms AS VARCHAR(MAX)))
    , CONVERT(INT,CAST(bedrooms AS VARCHAR(MAX)))
    , CONVERT(INT,CAST(beds AS VARCHAR(MAX)))
    , CAST(amenities AS VARCHAR(MAX))
    , CONVERT(INT,TRIM(REPLACE(REPLACE(REPLACE(CAST(price AS VARCHAR(MAX)),'$',''),'.00',''),',','')))
    , CONVERT(INT,CAST(availability_365 AS VARCHAR(MAX)))
    , CONVERT(INT,CAST(number_of_reviews AS VARCHAR(MAX)))
    , CONVERT(FLOAT,CAST(review_scores_rating AS VARCHAR(MAX)))
    FROM [airbnb].[dbo].[scraped_listings]

(36724 rows affected)

Total execution time: 00:00:01.330

Create a table that lists out unique amenities from the 'listings.amenities\_concat' field

In [14]:
CREATE TABLE [airbnb].[dbo].[amenities]
(amenity_name VARCHAR(MAX));

INSERT INTO [airbnb].[dbo].[amenities]
SELECT DISTINCT
TRIM(LOWER(value))
FROM [airbnb].[dbo].[listings]
CROSS APPLY STRING_SPLIT(amenities_concat,',')

(1623 rows affected)

Total execution time: 00:00:01.977

Create a list of unique host verifications

In [15]:
CREATE TABLE [airbnb].[dbo].[host_verifications]
(form_of_verification VARCHAR(MAX));

INSERT INTO [airbnb].[dbo].[host_verifications]
SELECT DISTINCT
TRIM(LOWER(value))
FROM [airbnb].[dbo].[hosts]
CROSS APPLY STRING_SPLIT(verifications,',')

(63 rows affected)

Total execution time: 00:00:00.207

Create a table that creates a unique record for each singular amenity at each listing

In [16]:
CREATE TABLE [airbnb].[dbo].[listings_by_amenity]
(listing_id INT
, form_of_verification VARCHAR(MAX));

INSERT INTO [airbnb].[dbo].[listings_by_amenity]
SELECT DISTINCT
listings.listing_id
, amenities.amenity_name
FROM [airbnb].[dbo].[listings] listings
CROSS JOIN [airbnb].[dbo].[amenities] amenities
WHERE LOWER(listings.amenities_concat) LIKE CONCAT('%',LOWER(amenities.amenity_name),'%')

(962250 rows affected)

Total execution time: 00:06:09.320

In [17]:
CREATE TABLE [airbnb].[dbo].[hosts_by_verification]
(listing_id INT
, amenity_nane VARCHAR(MAX));

INSERT INTO [airbnb].[dbo].[hosts_by_verification]
SELECT DISTINCT
hosts.host_id
, host_verifications.form_of_verification
FROM [airbnb].[dbo].[hosts] hosts
CROSS JOIN [airbnb].[dbo].[host_verifications] host_verifications
WHERE LOWER(hosts.verifications) LIKE CONCAT('%',LOWER(host_verifications.form_of_verification),'%')

(340386 rows affected)

Total execution time: 00:00:06.529